In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.quantization import get_default_qconfig, prepare, convert, fuse_modules
from data_loader import test_dataset
from models.ResNet import get_resnet18_model  # <- Adjust this path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Load your custom model with 3 input channels
model = get_resnet18_model(num_classes=2, in_channels=3)
model.eval()

# 2. Fuse modules (standard for ResNet18)
fuse_list = [
    ['conv1', 'bn1', 'relu'],
    ['layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu'],
    ['layer1.0.conv2', 'layer1.0.bn2'],
    ['layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu'],
    ['layer1.1.conv2', 'layer1.1.bn2'],

    ['layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu'],
    ['layer2.0.conv2', 'layer2.0.bn2'],
    ['layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu'],
    ['layer2.1.conv2', 'layer2.1.bn2'],

    ['layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu'],
    ['layer3.0.conv2', 'layer3.0.bn2'],
    ['layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu'],
    ['layer3.1.conv2', 'layer3.1.bn2'],

    ['layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu'],
    ['layer4.0.conv2', 'layer4.0.bn2'],
    ['layer4.1.conv1', 'layer4.1.bn1', 'layer4.1.relu'],
    ['layer4.1.conv2', 'layer4.1.bn2'],
]



Training set size: 960
Test set size: 240


/workspace/DR_Training/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/workspace/DR_Training/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
fuse_modules(model, fuse_list, inplace=True)

# 3. Set quantization config
model.qconfig = get_default_qconfig('fbgemm')

# 4. Prepare model (inserts observers)
prepare(model, inplace=True)

# 4. Prepare model (inserts observers)
prepare(model, inplace=True)

calibration_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
# 🔁 Move model to CPU (required for quantization)
model.to('cpu')

# 5. Calibrate using CPU inputs (match model device)
with torch.no_grad():
    for images, _ in calibration_loader:
        images = images.to('cpu')  # 🔁 Not to device — use CPU!
        model(images)
        break  # Use more batches for better calibration

# 5. Calibrate on a few batches of your test dataset


# 6. Convert to quantized model
quantized_model = convert(model, inplace=False)

# 7. Save
torch.save(quantized_model.state_dict(), "resnet18_quantized.pth")

print("✅ Quantized model saved to 'resnet18_quantized.pth'")

/workspace/DR_Training/.venv/lib/python3.10/site-packages/torch/ao/quantization/observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


✅ Quantized model saved to 'resnet18_quantized.pth'


In [5]:
import torch
from models.ResNet import get_resnet18_model
from torch.quantization import fuse_modules, get_default_qconfig, prepare, convert
from torch.utils.data import DataLoader
from data_loader import test_dataset

# Load and fuse
model = get_resnet18_model(num_classes=2, in_channels=3)
model.eval()

fuse_list = [
    ['conv1', 'bn1', 'relu'],
    ['layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu'],
    ['layer1.0.conv2', 'layer1.0.bn2'],
    ['layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu'],
    ['layer1.1.conv2', 'layer1.1.bn2'],

    ['layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu'],
    ['layer2.0.conv2', 'layer2.0.bn2'],
    ['layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu'],
    ['layer2.1.conv2', 'layer2.1.bn2'],

    ['layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu'],
    ['layer3.0.conv2', 'layer3.0.bn2'],
    ['layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu'],
    ['layer3.1.conv2', 'layer3.1.bn2'],

    ['layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu'],
    ['layer4.0.conv2', 'layer4.0.bn2'],
    ['layer4.1.conv1', 'layer4.1.bn1', 'layer4.1.relu'],
    ['layer4.1.conv2', 'layer4.1.bn2'],
]
fuse_modules(model, fuse_list, inplace=True)

# Prepare for quantization
model.qconfig = get_default_qconfig('fbgemm')
prepare(model, inplace=True)

# 🔁 Calibrate with real data
calibration_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
with torch.no_grad():
    for images, _ in calibration_loader:
        model(images.to("cpu"))  # must run on CPU
        break  # use more batches for better calibration

# Convert model
quantized_model = convert(model, inplace=False)

# Load previously saved quantized weights
quantized_model.load_state_dict(torch.load("saved_models/resnet18_quantized.pth", map_location="cpu"))

quantized_model.eval()
quantized_model.to("cpu")

print("✅ Quantized model loaded and calibrated successfully.")


/workspace/DR_Training/.venv/lib/python3.10/site-packages/torch/ao/quantization/observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


✅ Quantized model loaded and calibrated successfully.


/workspace/DR_Training/.venv/lib/python3.10/site-packages/torch/_utils.py:446: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


In [6]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, roc_auc_score
from torch.utils.data import DataLoader
from data_loader import test_dataset

# # ⚠️ Ensure your quantized model is loaded and on CPU
# model.eval()
# model.to("cpu")  # Just to be extra sure

# Load test set
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

# Collect predictions
all_probs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to("cpu")     # <- ⚠️ match model device
        labels = labels.to("cpu")
        outputs = quantized_model(inputs)
        probs = torch.softmax(outputs, dim=1)[:, 1]  # Class 1 = unhealthy
        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_probs = np.array(all_probs)
all_labels = np.array(all_labels)

# Compute sensitivity & specificity
def compute_sens_spec(threshold):
    preds = (all_probs > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(all_labels, preds, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    return sensitivity, specificity



NotImplementedError: Could not run 'quantized::conv2d_relu.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d_relu.new' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMTIA, AutogradMeta, Tracer, AutocastCPU, AutocastMTIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Met

In [ ]:
# Compute AUC
auc = roc_auc_score(all_labels, all_probs)
print(f"AUC: {auc:.3f}")

# Try a threshold
threshold = 0.9
sens, spec = compute_sens_spec(threshold)
print(f"Threshold: {threshold:.2f} | Sensitivity: {sens:.3f} | Specificity: {spec:.3f}")
